<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/tensorrt_torchtrt_efficientnet/nvidia_logo.png" width="90px">

# PySpark Huggingface Inferencing
### Conditional generation with Tensorflow

In this notebook, we demonstrate distributed inference with the T5 transformer to perform sentence translation.  
From: https://huggingface.co/docs/transformers/model_doc/t5

Note that cuFFT/cuDNN/cuBLAS registration errors are expected (as of `tf=2.17.0`) and will not affect behavior, as noted in [this issue.](https://github.com/tensorflow/tensorflow/issues/62075)

In [1]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

# Manually enable Huggingface tokenizer parallelism to avoid disabling with PySpark parallelism.
# See (https://github.com/huggingface/transformers/issues/5486) for more info. 
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

2025-02-04 13:53:50.831324: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-04 13:53:50.838528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-04 13:53:50.846226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-04 13:53:50.848585: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 13:53:50.854859: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import tensorflow as tf

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        
print(tf.__version__)

2.17.0


I0000 00:00:1738706031.770264 3625306 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738706031.793270 3625306 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738706031.796251 3625306 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [3]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

task_prefix = "translate English to German: "

lines = [
    "The house is wonderful",
    "Welcome to NYC",
    "HuggingFace is a company"
]

input_sequences = [task_prefix + l for l in lines]

I0000 00:00:1738706032.132191 3625306 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738706032.134996 3625306 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738706032.137528 3625306 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1738706032.251302 3625306 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [4]:
inputs = tokenizer(input_sequences, 
                      padding=True,
                      return_tensors="tf")
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=128)

I0000 00:00:1738706033.555987 3625654 service.cc:146] XLA service 0x712d300025f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738706033.556005 3625654 service.cc:154]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-02-04 13:53:53.558887: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-04 13:53:53.569767: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1738706033.604327 3625654 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [5]:
[tokenizer.decode(o, skip_special_tokens=True) for o in outputs]

['Das Haus ist wunderbar',
 'Willkommen in NYC',
 'HuggingFace ist ein Unternehmen']

## PySpark

In [6]:
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, col, struct
from pyspark.ml.functions import predict_batch_udf

In [7]:
import json
import pandas as pd
import datasets
from datasets import load_dataset
datasets.disable_progress_bars()

Check the cluster environment to handle any platform-specific Spark configurations.

In [8]:
on_databricks = os.environ.get("DATABRICKS_RUNTIME_VERSION", False)
on_dataproc = os.environ.get("DATAPROC_IMAGE_VERSION", False)
on_standalone = not (on_databricks or on_dataproc)

#### Create Spark Session

For local standalone clusters, we'll connect to the cluster and create the Spark Session.  
For CSP environments, Spark will either be preconfigured (Databricks) or we'll need to create the Spark Session (Dataproc).

In [9]:
conf = SparkConf()

if 'spark' not in globals():
    if on_standalone:
        import socket
        
        conda_env = os.environ.get("CONDA_PREFIX")
        hostname = socket.gethostname()
        conf.setMaster(f"spark://{hostname}:7077")
        conf.set("spark.pyspark.python", f"{conda_env}/bin/python")
        conf.set("spark.pyspark.driver.python", f"{conda_env}/bin/python")
    elif on_dataproc:
        conf.set("spark.executorEnv.TF_GPU_ALLOCATOR", "cuda_malloc_async")

    conf.set("spark.executor.cores", "8")
    conf.set("spark.task.resource.gpu.amount", "0.125")
    conf.set("spark.executor.resource.gpu.amount", "1")
    conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
    conf.set("spark.python.worker.reuse", "true")

conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1000")
spark = SparkSession.builder.appName("spark-dl-examples").config(conf=conf).getOrCreate()
sc = spark.sparkContext

25/02/04 13:53:54 WARN Utils: Your hostname, cb4ae00-lcedt resolves to a loopback address: 127.0.1.1; using 10.110.47.100 instead (on interface eno1)
25/02/04 13:53:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/04 13:53:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Load the IMBD Movie Reviews dataset from Huggingface.

In [10]:
dataset = load_dataset("imdb", split="test")
dataset = dataset.to_pandas().drop(columns="label")

### Create PySpark DataFrame

In [11]:
df = spark.createDataFrame(dataset).repartition(8)
df.schema

StructType([StructField('text', StringType(), True)])

In [12]:
df.count()

25000

In [13]:
df.take(1)

25/02/04 13:54:01 WARN TaskSetManager: Stage 6 contains a task of very large size (4021 KiB). The maximum recommended task size is 1000 KiB.


[Row(text="Anyone remember the first CKY, CKY2K etc..? Back when it was about making crazy cool stuff, rather than watching Bam Margera act like a douchebag, spoiled 5 year old, super/rock-star wannabe.<br /><br />The show used to be awesome, however, Bam's fame and wealth has led him to believe, that we now enjoy him acting childish and idiotic, more than actual cool stuff, that used to be in ex. CKY2K.<br /><br />The acts are so repetitive, there's like nothing new, except annoying stupidity and rehearsed comments... The only things we see is Bam Margera, so busy showing us how much he doesn't care, how much money he got or whatsoever.<br /><br />I really got nothing much left to say except, give us back CKY2K, cause Bam suck..<br /><br />I enjoy watching Steve-o, Knoxville etc. a thousand times more.")]

### Save the test dataset as parquet files

In [14]:
data_path = "spark-dl-datasets/imdb_test"
if on_databricks:
    dbutils.fs.mkdirs("/FileStore/spark-dl-datasets")
    data_path = "dbfs:/FileStore/" + data_path

df.write.mode("overwrite").parquet(data_path)

25/02/04 13:54:02 WARN TaskSetManager: Stage 9 contains a task of very large size (4021 KiB). The maximum recommended task size is 1000 KiB.


#### Load and preprocess DataFrame

Define our preprocess function. We'll take the first sentence from each sample as our input for translation.

In [15]:
def preprocess(text: pd.Series, prefix: str = "") -> pd.Series:
    @pandas_udf("string")
    def _preprocess(text: pd.Series) -> pd.Series:
        return pd.Series([prefix + s.split(".")[0] for s in text])
    return _preprocess(text)

In [16]:
# Limit to N rows, since this can be slow
df = spark.read.parquet(data_path).limit(256).repartition(8)
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|Doesn't anyone bother to check where this kind of sludge comes from before blathering on about it...|
|There were two things I hated about WASTED : The directing and the script . I know I`m opening my...|
|I'm rather surprised that anybody found this film touching or moving.<br /><br />The basic premis...|
|Cultural Vandalism Is the new Hallmark production of Gulliver's Travels an act of cultural vandal...|
|I was at Wrestlemania VI in Toronto as a 10 year old, and the event I saw then was pretty differe...|
|This movie has been done before. It is basically a unoriginal combo of "Napoleon Dynamite" and "S...|
|[ as a new resolution for this year 2005, i decide to write a comment fo

Append a prefix to tell the model to translate English to French:

In [17]:
input_df = df.select(preprocess(col("text"), "translate English to French: ").alias("input")).cache()
input_df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                               input|
+----------------------------------------------------------------------------------------------------+
|translate English to French: Doesn't anyone bother to check where this kind of sludge comes from ...|
|translate English to French: There were two things I hated about WASTED : The directing and the s...|
|   translate English to French: I'm rather surprised that anybody found this film touching or moving|
|translate English to French: Cultural Vandalism Is the new Hallmark production of Gulliver's Trav...|
|translate English to French: I was at Wrestlemania VI in Toronto as a 10 year old, and the event ...|
|                                        translate English to French: This movie has been done before|
|translate English to French: [ as a new resolution for this year 2005, i

## Inference using Spark DL API

Distributed inference using the PySpark [predict_batch_udf](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.functions.predict_batch_udf.html#pyspark.ml.functions.predict_batch_udf):

- predict_batch_fn uses Tensorflow APIs to load the model and return a predict function which operates on numpy arrays 
- predict_batch_udf will convert the Spark DataFrame columns into numpy input batches for the predict function

In [18]:
def predict_batch_fn():
    import tensorflow as tf
    import numpy as np
    from transformers import TFT5ForConditionalGeneration, AutoTokenizer

    # Enable GPU memory growth
    print("initializing model")
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

    model = TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
    tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")

    def predict(inputs):
        flattened = np.squeeze(inputs).tolist()
        inputs = tokenizer(flattened, 
                            padding=True, 
                            return_tensors="tf")
        outputs = model.generate(input_ids=inputs["input_ids"],
                                 attention_mask=inputs["attention_mask"],
                                 max_length=128)
        string_outputs = np.array([tokenizer.decode(o, skip_special_tokens=True) for o in outputs])
        print("predict: {}".format(len(flattened)))
        return string_outputs
    
    return predict

In [19]:
generate = predict_batch_udf(predict_batch_fn,
                             return_type=StringType(),
                             batch_size=32)

In [20]:
%%time
# first pass caches model/fn
preds = input_df.withColumn("preds", generate(struct("input")))
results = preds.collect()

CPU times: user 9.07 ms, sys: 8.83 ms, total: 17.9 ms
Wall time: 19.3 s


In [21]:
%%time
preds = input_df.withColumn("preds", generate("input"))
results = preds.collect()

CPU times: user 7.51 ms, sys: 4.96 ms, total: 12.5 ms
Wall time: 12.4 s


In [22]:
%%time
preds = input_df.withColumn("preds", generate(col("input")))
results = preds.collect()

CPU times: user 5.46 ms, sys: 5.98 ms, total: 11.4 ms
Wall time: 11.4 s


In [23]:
preds.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                             input|                                             preds|
+--------------------------------------------------+--------------------------------------------------+
|translate English to French: Doesn't anyone bot...|Ne s'ennuie-t-il pas de vérifier où viennent ce...|
|translate English to French: There were two thi...|Il y avait deux choses que j'ai hâte de voir : ...|
|translate English to French: I'm rather surpris...|Je suis plutôt surpris que quelqu'un ait trouvé...|
|translate English to French: Cultural Vandalism...|Vandalisme culturel La nouvelle production Hall...|
|translate English to French: I was at Wrestlema...|J'étais à Wrestlemania VI à Toronto en 10 ans, ...|
|translate English to French: This movie has bee...|                       Ce film a été réalisé avant|
|translate English to French: [ as a new resolut...|[ en tant qu

In [24]:
input_df2 = df.select(preprocess(col("text"), "translate English to German: ").alias("input")).cache()

In [25]:
%%time
# first pass caches model/fn
preds = input_df2.withColumn("preds", generate(struct("input")))
result = preds.collect()

CPU times: user 9.1 ms, sys: 4.04 ms, total: 13.1 ms
Wall time: 14.9 s


In [26]:
%%time
preds = input_df2.withColumn("preds", generate("input"))
result = preds.collect()

CPU times: user 6.62 ms, sys: 5.23 ms, total: 11.9 ms
Wall time: 11.9 s


In [27]:
%%time
preds = input_df2.withColumn("preds", generate(col("input")))
result = preds.collect()

CPU times: user 8.67 ms, sys: 3.27 ms, total: 11.9 ms
Wall time: 11.7 s


In [28]:
preds.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                             input|                                             preds|
+--------------------------------------------------+--------------------------------------------------+
|translate English to German: Doesn't anyone bot...|Warum hat man sich nicht angeschaut, woher der ...|
|translate English to German: There were two thi...|Es gab zwei Dinge, die ich hat an WASTED gehass...|
|translate English to German: I'm rather surpris...|Ich bin ziemlich überrascht, dass jemand diesen...|
|translate English to German: Cultural Vandalism...|Kultureller Vandalismus Ist die neue Hallmark-P...|
|translate English to German: I was at Wrestlema...|Ich war als 10 Jahre alt bei Wrestlemania VI in...|
|translate English to German: This movie has bee...|             Dieser Film wurde bereits vorgenommen|
|translate English to German: [ as a new resolut...|[ als neue E

## Using Triton Inference Server
In this section, we demonstrate integration with the [Triton Inference Server](https://developer.nvidia.com/nvidia-triton-inference-server), an open-source, GPU-accelerated serving solution for DL.  
We use [PyTriton](https://github.com/triton-inference-server/pytriton), a Flask-like framework that handles client/server communication with the Triton server.  

The process looks like this:
- Distribute a PyTriton task across the Spark cluster, instructing each node to launch a Triton server process.
- Define a Triton inference function, which contains a client that binds to the local server on a given node and sends inference requests.
- Wrap the Triton inference function in a predict_batch_udf to launch parallel inference requests using Spark.
- Finally, distribute a shutdown signal to terminate the Triton server processes on each node.

<img src="../images/spark-server.png" alt="drawing" width="700"/>

In [29]:
from functools import partial

Import the helper class from server_utils.py:

In [30]:
sc.addPyFile("server_utils.py")

from server_utils import TritonServerManager

Define the Triton Server function:

In [31]:
def triton_server(ports):
    import time
    import signal
    import numpy as np
    import tensorflow as tf
    from transformers import TFT5ForConditionalGeneration, AutoTokenizer
    from pytriton.decorators import batch
    from pytriton.model_config import DynamicBatcher, ModelConfig, Tensor
    from pytriton.triton import Triton, TritonConfig
    from pyspark import TaskContext

    print(f"SERVER: Initializing Conditional Generation model on worker {TaskContext.get().partitionId()}.")

    # Enable GPU memory growth
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)
    
    tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
    model = TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

    @batch
    def _infer_fn(**inputs):
        sentences = np.squeeze(inputs["text"]).tolist()
        print(f"SERVER: Received batch of size {len(sentences)}")
        decoded_sentences = [s.decode("utf-8") for s in sentences]
        inputs = tokenizer(decoded_sentences,
                            padding=True,
                            return_tensors="tf")
        output_ids = model.generate(input_ids=inputs["input_ids"],
                                    attention_mask=inputs["attention_mask"],
                                    max_length=128)
        outputs = np.array([[tokenizer.decode(o, skip_special_tokens=True)] for o in output_ids])
        return {
            "translations": outputs,
        }

    workspace_path = f"/tmp/triton_{time.strftime('%m_%d_%M_%S')}"
    triton_conf = TritonConfig(http_port=ports[0], grpc_port=ports[1], metrics_port=ports[2])
    with Triton(config=triton_conf, workspace=workspace_path) as triton:
        triton.bind(
            model_name="ConditionalGeneration",
            infer_func=_infer_fn,
            inputs=[
                Tensor(name="text", dtype=object, shape=(-1,)),
            ],
            outputs=[
                Tensor(name="translations", dtype=object, shape=(-1,)),
            ],
            config=ModelConfig(
                max_batch_size=64,
                batcher=DynamicBatcher(max_queue_delay_microseconds=5000),  # 5ms
            ),
            strict=True,
        )

        def _stop_triton(signum, frame):
            # The server manager sends SIGTERM to stop the server; this function ensures graceful cleanup.
            print("SERVER: Received SIGTERM. Stopping Triton server.")
            triton.stop()

        signal.signal(signal.SIGTERM, _stop_triton)

        print("SERVER: Serving inference")
        triton.serve()

#### Start Triton servers

The `TritonServerManager` will handle the lifecycle of Triton server instances across the Spark cluster:
- Find available ports for HTTP/gRPC/metrics
- Deploy a server on each node via stage-level scheduling
- Gracefully shutdown servers across nodes

In [ ]:
model_name = "ConditionalGeneration"
server_manager = TritonServerManager(model_name=model_name)

In [ ]:
# Returns {'hostname', (server_pid, [http_port, grpc_port, metrics_port])}
server_manager.start_servers(triton_server)

2025-02-07 11:03:44,809 - INFO - Requesting stage-level resources: (cores=5, gpu=1.0)
2025-02-07 11:03:44,810 - INFO - Starting 1 servers.


{'cb4ae00-lcedt': (2020631, [7000, 7001, 7002])}

#### Define client function

Get the hostname -> url mapping from the server manager:

In [ ]:
host_to_http_url = server_manager.host_to_http_url  # or server_manager.host_to_grpc_url

Define the Triton inference function, which returns a predict function for batch inference through the server:

In [36]:
def triton_fn(model_name, host_to_url):
    import socket
    import numpy as np
    from pytriton.client import ModelClient

    url = host_to_url[socket.gethostname()]
    print(f"Connecting to Triton model {model_name} at {url}.")

    def infer_batch(inputs):
        with ModelClient(url, model_name, inference_timeout_s=240) as client:
            flattened = np.squeeze(inputs).tolist() 
            # Encode batch
            encoded_batch = [[text.encode("utf-8")] for text in flattened]
            encoded_batch_np = np.array(encoded_batch, dtype=np.bytes_)
            # Run inference
            result_data = client.infer_batch(encoded_batch_np)
            result_data = np.squeeze(result_data["translations"], -1)
            return result_data
        
    return infer_batch

In [40]:
generate = predict_batch_udf(partial(triton_fn, model_name=model_name, host_to_url=host_to_http_url),
                             return_type=StringType(),
                             input_tensor_shapes=[[1]],
                             batch_size=32)

#### Load and preprocess DataFrame

In [37]:
def preprocess(text: pd.Series, prefix: str = "") -> pd.Series:
    @pandas_udf("string")
    def _preprocess(text: pd.Series) -> pd.Series:
        return pd.Series([prefix + s.split(".")[0] for s in text])
    return _preprocess(text)

In [38]:
df = spark.read.parquet(data_path).limit(256).repartition(8)

In [39]:
input_df = df.select(preprocess(col("text"), "translate English to French: ").alias("input")).cache()

25/02/04 13:55:37 WARN CacheManager: Asked to cache already cached data.


#### Run Inference

In [41]:
%%time
# first pass caches model/fn
preds = input_df.withColumn("preds", generate(struct("input")))
results = preds.collect()

CPU times: user 10.8 ms, sys: 8.12 ms, total: 18.9 ms
Wall time: 30 s


In [42]:
%%time
preds = input_df.withColumn("preds", generate("input"))
results = preds.collect()

CPU times: user 7.23 ms, sys: 3.43 ms, total: 10.7 ms
Wall time: 21.2 s


In [43]:
%%time
preds = input_df.withColumn("preds", generate(col("input")))
results = preds.collect()

CPU times: user 2.81 ms, sys: 12.7 ms, total: 15.5 ms
Wall time: 22.3 s


In [44]:
preds.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                             input|                                             preds|
+--------------------------------------------------+--------------------------------------------------+
|translate English to French: Doesn't anyone bot...|Ne s'ennuie-t-il pas de vérifier où viennent ce...|
|translate English to French: There were two thi...|Il y avait deux choses que j'ai hâte de voir : ...|
|translate English to French: I'm rather surpris...|Je suis plutôt surpris que quelqu'un ait trouvé...|
|translate English to French: Cultural Vandalism...|Vandalisme culturel La nouvelle production Hall...|
|translate English to French: I was at Wrestlema...|J'étais à Wrestlemania VI à Toronto en 10 ans, ...|
|translate English to French: This movie has bee...|                       Ce film a été réalisé avant|
|translate English to French: [ as a new resolut...|[ en tant qu

#### Shut down server on each executor

In [45]:
server_manager.stop_servers()

2025-02-04 13:56:54,506 - INFO - Requesting stage-level resources: (cores=5, gpu=1.0)
2025-02-04 13:56:59,695 - INFO - Sucessfully stopped 1 servers.                 


[True]

In [46]:
if not on_databricks: # on databricks, spark.stop() puts the cluster in a bad state
    spark.stop()